In [1]:
import h5py as h5
data = h5.File("../../../Data/EC2_blocks_1_8_9_15_76_89_105_CV_HG_align_window_-0.5_to_0.79_file_nobaseline.h5")

In [2]:
import numpy as np
x = data['Xhigh gamma'][:]
y = data['y'][:]

In [3]:
import random
import numpy as np
def generateMoreExamples(x_set, totalExamples):
    n_original_examples = len(x_set)
    n_examples = n_original_examples
    new_examples = []
    sigma = 0.5
    while n_examples < totalExamples:
        # only use original examples to ensure we don't get closer and closer to the mean
        i = random.randint(0,n_original_examples-1)
        j = random.randint(0,n_original_examples-1)
        if j==i: j = (j+1)%n_original_examples
        mixing = random.uniform(0, 1)
        noise = np.random.randn(x_set[0].shape[0], x_set[0].shape[1])*sigma
        new_example = x_set[i]*mixing + x_set[j]*(1-mixing) + noise
        new_example = list(new_example.clip(min=0))
        new_examples.append(new_example)
        n_examples += 1
    return list(x_set) + new_examples

In [4]:
x_plus = []
y_plus = []
x_isolated = []
y_isolated = []
nIsolate = 5
for i in range(57):
    if i%10==0: print i
    indices = np.where(y==i)[0]
    x_isolated += list(x[indices[:nIsolate]])
    y_isolated += [i]*nIsolate
    new_x = generateMoreExamples(x[indices[nIsolate:]], 500)
    x_plus += new_x
    y_plus += [i]*len(new_x)
print "Finished generating new examples."
x = np.array(x_plus)
print "Finished converting x array."
y = np.array(y_plus)
print "Finished converting y array."
x_isolated = np.array(x_isolated)
y_isolated = np.array(y_isolated)
print "Finished converting isolated arrays."

0
10
20
30
40
50
Finished generating new examples.
Finished converting x array.
Finished converting y array.
Finished converting isolated arrays.


In [5]:
# To prevent a memory error, shuffle front and back halves separately
from sklearn.utils import shuffle
a = x[:15000]
b = y[:15000]
a, b = shuffle(a, b, random_state=0)
x[:15000] = a
y[:15000] = b
a = x[15000:]
b = y[15000:]
a, b = shuffle(a, b, random_state=0)
x[15000:] = a
y[15000:] = b

In [6]:
new_data = h5.File("../../../Data/Expanded_ECoG_285Isolated_GaussianNoise.h5", "w")
new_data.create_dataset('Xhigh gamma', data=x)
new_data.create_dataset('y', data=y)
new_data.create_dataset('Xhigh gamma isolated', data=x_isolated)
new_data.create_dataset('y isolated', data=y_isolated)
new_data.close()